In [11]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("tripdata.dat")
database = db.Database("tripdata",pandas)

# The Pandas data structure is available as database.data. Use all the
# Pandas functions to invesigate the database
#print(database.data.describe())

from headers import *

# Removing some observations can be done directly using pandas.
#remove = (((database.data.PURPOSE != 1) & (database.data.PURPOSE != 3)) | (database.data.CHOICE == 0))
#database.data.drop(database.data[remove].index,inplace=True)

# Here we use the "biogeme" way for backward compatibility
#exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
#database.remove(exclude)

ASC_AUTO_DRIVE = Beta('ASC_AUTO_DRIVE',0,None,None,0)
ASC_AUTO_PASS = Beta('ASC_AUTO_PASS',0,None,None,0)
ASC_METRO = Beta('ASC_METRO',0,None,None,0)
ASC_TRAIN = Beta('ASC_TRAIN',0,None,None,0)
ASC_WALK = Beta('ASC_WALK',0,None,None,1)

B_IVTT = Beta('B_IVTT',0,None,None,0)
B_SEX = Beta('B_SEX',0,None,None,0)
B_AGE = Beta('B_AGE',0,None,None,0)
B_EMP = Beta('B_EMP',0,None,None,0)

#SM_COST =  SM_CO   * (  GA   ==  0  ) 
#TRAIN_COST =  TRAIN_CO   * (  GA   ==  0  )

#CAR_AV_SP =  DefineVariable('CAR_AV_SP',CAR_AV  * (  SP   !=  0  ),database)
#TRAIN_AV_SP =  DefineVariable('TRAIN_AV_SP',TRAIN_AV  * (  SP   !=  0  ),database)

#TRAIN_TT_SCALED = DefineVariable('TRAIN_TT_SCALED',\
#                                 TRAIN_TT / 100.0,database)
#TRAIN_COST_SCALED = DefineVariable('TRAIN_COST_SCALED',\
#                                   TRAIN_COST / 100,database)
#SM_TT_SCALED = DefineVariable('SM_TT_SCALED', SM_TT / 100.0,database)
#SM_COST_SCALED = DefineVariable('SM_COST_SCALED', SM_COST / 100,database)
#CAR_TT_SCALED = DefineVariable('CAR_TT_SCALED', CAR_TT / 100,database)
#CAR_CO_SCALED = DefineVariable('CAR_CO_SCALED', CAR_CO / 100,database)

V1 = ASC_AUTO_DRIVE + \
     B_IVTT * ivtt1 + \
     B_EMP * otarms
V2 = ASC_AUTO_PASS + \
     B_IVTT * ivtt2 + \
     B_AGE * age + \
     B_EMP * otarms
V3 = ASC_METRO + \
     B_IVTT * ivtt3 + \
     B_EMP * otarms
V4 = ASC_TRAIN + \
     B_IVTT * ivtt4 + \
     B_EMP * otarms
V5 = ASC_WALK + \
     B_IVTT * ivtt5 + \
     B_SEX * female

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4,
     5: V5}

# Associate the availability conditions with the alternatives

av = {1: avail1,
      2: avail2,
      3: avail3,
      4: avail4,
      5: avail5}

logprob = bioLogLogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob) # This is my log likelihood function
biogeme.modelName = "A1Practice_WalkFixed"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

ASC_AUTO_DRIVE=	-0.228
ASC_AUTO_PASS=	-0.501
ASC_METRO=	0.672
ASC_TRAIN=	0.0576
B_AGE=	-0.02
B_EMP=	-0.279
B_IVTT=	-0.0398
B_SEX=	0.000222
                   Value      Std err        t-test   p-value  Rob. Std err  \
ASC_AUTO_DRIVE -0.228396     0.238564 -9.573796e-01  0.338376      0.234617   
ASC_AUTO_PASS  -0.501292     0.548131 -9.145489e-01  0.360429      0.611976   
ASC_METRO       0.671532     0.222047  3.024283e+00  0.002492      0.211741   
ASC_TRAIN       0.057579     0.267193  2.154967e-01  0.829380      0.248213   
B_AGE          -0.020027     0.018559 -1.079113e+00  0.280537      0.021667   
B_EMP          -0.278632    25.116025 -1.109380e-02  0.991149      0.017936   
B_IVTT         -0.039768     0.009989 -3.981403e+00  0.000069      0.008618   
B_SEX           0.000222  2132.702070  1.042498e-07  1.000000      1.414847   

                Rob. t-test  Rob. p-value  
ASC_AUTO_DRIVE    -0.973486      0.330312  
ASC_AUTO_PASS     -0.819138      0.412708  
ASC_METRO        